# Assignment 3: Logistics Network Design
### Data-Driven Decision Making
### José Pedro Pinto - up201603713

# Introduction

Optimization problems arise every day in almost every aspect of life. They range in complexity from what we would be considered intuitive to large scale intractable problems, for which no exact answer can be reasonably obtained. These can be seen, for example in personal finances, where we try to decide how, where and when to apply capital. In marketing, where we try to do the correct amount and type of marketing, to obtain the best profit. Or in personal life, where we decide how much of a product to buy at a time, to balance, convenience and risk of spoilage or waste.<br>

In this project we are going to tackle a particular case of optimization, facility location problems. There is a large variety of these, which range in complexity. For our particular case, we have three main components, airports, distribution centers (DC) and locations.<br>
We have three airports, which are always open, and whish supply our DCs. With the cost increasing, the further away the airport is from the DC.<br>
We have DCs, whose placements coincide with the locations, supplying said locations, once more, with the cost increasing with distance. There is a limit to the amount each DC can distribute also a cost for each open DC (those that transport something).<br>
Finally, we have the locations, for which the items will be transported. Each has its own demand, which must be fulfilled.<br>
We as such, want to reduce the cost, characterized by sum of the cost of opening the facilities, the cost of air transportation and the cost of land transportation.<br>

In [1]:
from amplpy import AMPL, DataFrame
import pandas as pd
import numpy as np
import math
import time
from sklearn.preprocessing import LabelEncoder

# Pre-Processing

First, we will read the data and see the values.

In [2]:
populationDF = pd.read_csv("PopulationContPT-2020.csv")
populationDF.head()

,asciiname,population,latitude,longitude
0,Lisbon,517802,38.71667,-9.13333
1,Porto,249633,41.14961,-8.61099
2,Amadora,178858,38.75382,-9.23083
3,Braga,121394,41.55032,-8.42005
4,Setubal,117110,38.52440,-8.88820


We can see in the above data, that it is not in the format or amounts required.<br>
We need to transform asciiname into a numerical identifier, population into demand and latitude, longitude into distances.<br>

Now, we will transform the population into the demand for each city. For each 50 population is 1 demand (rounded up).<br>

In [3]:
demandDF = populationDF
demandDF["demand"] = (populationDF["population"]*50/1000).apply(np.ceil).astype("int32")
demandDF = demandDF.drop("population",axis=1)

We will now transform the name of the city into a numerical identifier.<br>

In [4]:
labelDF = demandDF
le = LabelEncoder().fit(demandDF["asciiname"])
labelDF["locationLabel"] = le.transform(demandDF["asciiname"])
labelDF = labelDF.drop("asciiname",axis=1)

Below we obtain the distance values from the latitude and longitude.<br>

In [5]:
# earth's radius
r = 6371.009

Obtain distances in Km from origin point (min latitude, min longitude).<br>

In [6]:
distanceDF = labelDF
distanceDF["d1"] = 2*math.pi*r*labelDF["latitude"]/360
#remove the min to get smaller values
distanceDF["d1"] = distanceDF["d1"] - min(distanceDF["d1"])
distanceDF["d2"] = 2*math.pi*r*labelDF["longitude"]/360
#remove the min to get smaller values
distanceDF["d2"] = distanceDF["d2"] - min(distanceDF["d2"])
distanceDF = distanceDF.drop(["latitude","longitude"],axis=1)

Now, we show the full preprocessed data.<br>

In [7]:
distanceDF

,demand,locationLabel,d1,d2
0,25891,173,188.731416,34.868554
1,12482,261,459.262383,92.950194
2,8943,31,192.862313,24.027034
3,6070,74,503.819365,114.181784
4,5856,326,167.351937,62.125805
...,...,...,...,...
374,274,279,269.686996,103.712767
375,273,347,490.129026,127.757591
376,272,207,33.178389,99.143761
377,271,311,479.506560,109.533829


The data is in our inteded format and as such we will now procees with modeling.<br>

# AMPL

In this part we will use AMPL to solve our problem.<br>

### Initial Model

The first model we will present is the "base" model, which was the first one created, which is consistent with our objective and could obtain the best results.<br>
This model, is however, extremely inefficient and we will as such limit the processing to the few first locations. This was done in such a way that the processing takes about 60s (1 minute) in the used machine.<br>

In [8]:
df = distanceDF.iloc[0:75,:]

First, we will load our model and set the solver.<br>

In [9]:
ampl = AMPL()
ampl.option['solver'] = 'gurobi'
ampl.read("base.mod")

Now, we will set the constant and list parameters.<br>

In [10]:
ampl.param['i'] = 3
ampl.param['j'] = len(df)
ampl.param['k'] = len(df)
ampl.param['capacity'] = 50000
ampl.param['facilityCost'] = 100000
ampl.param['demand'] = df["demand"].tolist()

As the air transport distances are more complex to obtain, we set them now.<br>

In [11]:
airports = ["Maia","Moscavide e Portela","Faro"]
airDistances = []
# for each airport
for i in le.transform(airports).tolist():
    for j in range(0,len(df)):
        #obtain distance of facility to airport
        d1 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,2]-df.iloc[j,2])
        d2 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,3]-df.iloc[j,3])
        airDistances.append(d1+d2) 

And, finally, we obtain the land transport costs.<br>

In [12]:
landDistances = []
for i in range(0,len(df)):
    for j in range(0,len(df)):
        d1 = abs(df.iloc[i,2]-df.iloc[j,2])
        d2 = abs(df.iloc[i,3]-df.iloc[j,3])
        landDistances.append(d1+d2) 

And, add them to our model.<br>

In [13]:
ampl.param['airCost'] = [0.001 * distance for distance in airDistances]
ampl.param['landCost'] = [0.01 * distance for distance in landDistances]

Now we will use the model to solve the restricted problem.<br>

In [14]:
start = time.time()
ampl.solve()
end = time.time()
print(end - start, " seconds taken")

Gurobi 9.0.0: optimal solution; objective 508904.572
88125 simplex iterations
2079 branch-and-cut nodes
plus 315 simplex iterations for intbasis
8.290882587432861  seconds taken


Finally, we will show the results for the facilities and amounts transported.<br>

In [15]:
print("Facilities - Inactive one not shown.")
x = ampl.getVariable("facility")
for index, xi in x:
    if (xi.value()!=0):
        print(index,xi.value())
print("Air Transports - Inactive one not shown.")
x = ampl.getVariable("airTransport")
for index, xi in x:
    if (xi.value()!=0):
        print(index,xi.value())
print("Land Transports - Inactive one not shown.")
x = ampl.getVariable("transport")
for index, xi in x:
    if (xi.value()!=0):
        print(index,xi.value())

Facilities - Inactive one not shown.
1.0 1.0
6.0 1.0
7.0 1.0
67.0 1.0
Air Transports - Inactive one not shown.
(1.0, 6.0) 47413.999999999985
(1.0, 67.0) 50000.0
(2.0, 1.0) 50000.0
(2.0, 7.0) 50000.000000000015
Land Transports - Inactive one not shown.
(1.0, 1.0) 25891.0
(1.0, 5.0) 2967.0000000000155
(1.0, 17.0) 2629.0
(1.0, 18.0) 2626.0
(1.0, 19.0) 2564.0
(1.0, 27.0) 2057.0
(1.0, 40.0) 1701.0
(1.0, 41.0) 1690.0
(1.0, 46.0) 962.9999999999854
(1.0, 47.0) 1508.0
(1.0, 48.0) 1501.0
(1.0, 55.0) 1446.0
(1.0, 68.0) 1281.0
(1.0, 74.0) 1176.0
(6.0, 5.0) 2888.9999999999845
(6.0, 6.0) 5330.0
(6.0, 12.0) 2904.0
(6.0, 13.0) 2781.0
(6.0, 16.0) 2709.0
(6.0, 20.0) 2500.0
(6.0, 23.0) 2330.0
(6.0, 24.0) 2256.0
(6.0, 26.0) 2068.0
(6.0, 28.0) 2031.0
(6.0, 30.0) 1947.0
(6.0, 31.0) 1925.0
(6.0, 37.0) 1738.0
(6.0, 39.0) 1011.0
(6.0, 42.0) 1673.9999999999998
(6.0, 45.0) 1606.0
(6.0, 50.0) 1470.0
(6.0, 51.0) 1462.0
(6.0, 54.0) 1446.0
(6.0, 56.0) 1425.9999999999998
(6.0, 62.0) 1369.0
(6.0, 64.0) 1319.0
(6.0, 69

### Optimization 1

After the poor results of the "base" model, an attempt was made to create an optimized version, for which the result space is equivalent, meaning that it will still achieve the best result.<br>

A large number of optimizations was attempted, however, only two had noticeable improvements.<br>
The first one, was noting that, under no circumstance would a DC receive supply from an airport other than the closest one. This comes due to the unlimited capacity of airports and constant cost per unit. This as such allows us to pre calculate these values; add them to the cost of land transportation (from DC to customer) and get rid of the air cost parameter.<br>
The second improvement was a restriction on the number of facilities. With said number being at least the required amount to fulfill demand and, at most, the maximum number of facilities, such that, the total transport costs exceed the cost of all facilities.<br>

Even with these improvements, we still cannot obtain the results for the full data in a reasonable amount of time.<br>

In [16]:
df = distanceDF.iloc[0:100,:]

In [17]:
ampl = AMPL()
ampl.option['solver'] = 'gurobi'
ampl.read("optimized.mod")

No longer has the i (number of airports) parameter.<br>

In [18]:
ampl.param['j'] = len(df)
ampl.param['k'] = len(df)
ampl.param['capacity'] = 50000
ampl.param['facilityCost'] = 100000
ampl.param['demand'] = df["demand"].tolist()

In [19]:
airports = ["Maia","Moscavide e Portela","Faro"]
airDistances = []
for j in range(0,len(df)):
    ds = []
    for i in le.transform(airports).tolist():
        d1 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,2]-df.iloc[j,2])
        d2 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,3]-df.iloc[j,3])
        ds.append(d1+d2)
    airDistances.append(min(ds)) 

Calculate the cost of transport, taking into account, the air and land costs.<br>

In [20]:
airCost = 0.001
landCost = 0.01

transportCost = []
for i in range(0,len(df)):
    for j in range(0,len(df)):
        d1 = abs(df.iloc[i,2]-df.iloc[j,2])
        d2 = abs(df.iloc[i,3]-df.iloc[j,3])
        transportCost.append((d1+d2)*landCost+airDistances[i]*airCost) 

In [21]:
ampl.param['transportCost'] = transportCost

Train the new model.<br>

In [22]:
start = time.time()
ampl.solve()
end = time.time()
print(end - start, " seconds taken")

Gurobi 9.0.0: optimal solution; objective 602299.6308
306240 simplex iterations
5718 branch-and-cut nodes
plus 296 simplex iterations for intbasis
33.221580028533936  seconds taken


In [23]:
print("Facilities - Inactive ones not shown.")
x = ampl.getVariable("facility")
for index, xi in x:
    if (xi.value()!=0):
        print(index,xi.value())
print("Land Transports - Inactive ones not shown.")
x = ampl.getVariable("transport")
for index, xi in x:
    if (round(xi.value())!=0):
        print(index,round(xi.value()))

Facilities - Inactive ones not shown.
1.0 1.0
2.0 1.0
4.0 1.0
7.0 1.0
74.0 1.0
Land Transports - Inactive ones not shown.
(1.0, 1.0) 25891
(1.0, 11.0) 3312
(1.0, 15.0) 1039
(1.0, 17.0) 2629
(1.0, 18.0) 2626
(1.0, 40.0) 1701
(1.0, 41.0) 1690
(1.0, 46.0) 1591
(1.0, 49.0) 1489
(1.0, 55.0) 1446
(1.0, 62.0) 1369
(1.0, 79.0) 1125
(1.0, 84.0) 1059
(1.0, 86.0) 1050
(1.0, 92.0) 1009
(1.0, 96.0) 974
(2.0, 2.0) 12482
(2.0, 6.0) 5146
(2.0, 9.0) 3541
(2.0, 16.0) 2709
(2.0, 21.0) 2499
(2.0, 23.0) 2330
(2.0, 29.0) 1971
(2.0, 34.0) 1822
(2.0, 44.0) 1640
(2.0, 45.0) 1606
(2.0, 53.0) 1447
(2.0, 56.0) 1426
(2.0, 58.0) 1404
(2.0, 59.0) 1385
(2.0, 60.0) 1385
(2.0, 63.0) 1322
(2.0, 67.0) 1297
(2.0, 70.0) 1221
(2.0, 71.0) 1196
(2.0, 80.0) 1104
(2.0, 83.0) 1067
(4.0, 4.0) 6070
(4.0, 6.0) 184
(4.0, 12.0) 2904
(4.0, 25.0) 2234
(4.0, 28.0) 2031
(4.0, 30.0) 1947
(4.0, 35.0) 1808
(4.0, 38.0) 1732
(4.0, 39.0) 1719
(4.0, 52.0) 1453
(4.0, 61.0) 1383
(4.0, 64.0) 1319
(4.0, 78.0) 1143
(4.0, 82.0) 1084
(4.0, 88.0) 1035


The results obtained have a few rounding errors. Unfortunately, any attempts to correct them directly in the model, either did not work or resulted in far worse performance. As such, we are rounding them at display time, which in very rare cases, could provide wrong values. In practice this was never observed.<br>

### Greedy

Having tried dozens of possible optimizations with no extra improvements, we have decided to try a new, approximated, approach.<br>
This approach is greedy search, where on each step, we select the best location. This is done by forcing the number of facilities to be 1 and the capacity usage to be maximum for all but the last facility chosen.<br>
Like for the optimized model, we pre calculate the total transport cost.<br>

In [24]:
df = distanceDF

In [25]:
airports = ["Maia","Moscavide e Portela","Faro"]
airDistances = []
for j in range(0,len(df)):
    ds = []
    for i in le.transform(airports).tolist():
        d1 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,2]-df.iloc[j,2])
        d2 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,3]-df.iloc[j,3])
        ds.append(d1+d2)
    airDistances.append(min(ds)) 

In [26]:
airCost = 0.001
landCost = 0.01

transportCost = []
for i in range(0,len(df)):
    for j in range(0,len(df)):
        d1 = abs(df.iloc[i,2]-df.iloc[j,2])
        d2 = abs(df.iloc[i,3]-df.iloc[j,3])
        transportCost.append((d1+d2)*landCost+airDistances[i]*airCost) 

For the greedy method, first we obtain the facilities, one by one, up to the point where remaining demand is no longer greater than a facility capacity.<br>

In [27]:
totalCost = 0
demand = df["demand"].tolist()
facilities = []
totalTransport = []
start = time.time()
for i in range(0,math.floor(sum(df["demand"].tolist())/50000)):
    # model needs to be reset after each itertion
    ampl = AMPL()
    ampl.option['solver'] = 'gurobi'
    ampl.read("greedy.mod")
    
    #set the parameters
    ampl.param['j'] = len(df)
    ampl.param['k'] = len(df)
    ampl.param['capacity'] = 50000
    ampl.param['facilityCost'] = 100000
    #only demand and transportCost change in each iteration
    ampl.param['demand'] = demand
    ampl.param['transportCost'] = transportCost
    #solve
    ampl.solve()
    
    # add new cost to the total cost
    totalCost += ampl.getObjective('cost').value()
    # get the facility list
    x = ampl.getVariable("facility")
    for index, xi in x:
        # if it is the selected facility
        if (xi.value()!=0):
            # save it
            facilities.append(index)
            break
    # get the thransport values
    x = ampl.getVariable("transport")
    for index, xi in x:
        # if something got transported
        if (round(xi.value())!=0):
            # reduce demand in tranport location -- ampl index starts at 1 but python at 0, reduce by 1
            demand[int(index[1])-1] -= round(xi.value()) #rounding needed due to rounding errors causing problems
            totalTransport.append((index,round(xi.value())))
    # for all transport combinations
    for i in range(0,len(df)):
        for j in range(0,len(df)):
            # if row is from the last selected facility
            if i == facilities[-1]-1:
                # change transport cost so that it will no longer transport
                transportCost[i*len(df)+j] = 99999

Gurobi 9.0.0: optimal solution; objective 102895.6535
379 simplex iterations
1 branch-and-cut nodes
plus 24 simplex iterations for intbasis
Gurobi 9.0.0: optimal solution; objective 104326.3062
379 simplex iterations
1 branch-and-cut nodes
plus 41 simplex iterations for intbasis
Gurobi 9.0.0: optimal solution; objective 105196.0446
380 simplex iterations
1 branch-and-cut nodes
plus 29 simplex iterations for intbasis
Gurobi 9.0.0: optimal solution; objective 117828.0851
387 simplex iterations
1 branch-and-cut nodes
plus 133 simplex iterations for intbasis
Gurobi 9.0.0: optimal solution; objective 118854.0156
394 simplex iterations
1 branch-and-cut nodes
plus 57 simplex iterations for intbasis
Gurobi 9.0.0: optimal solution; objective 140595.1419
397 simplex iterations
1 branch-and-cut nodes
plus 69 simplex iterations for intbasis
Gurobi 9.0.0: optimal solution; objective 207334.1627
2151 simplex iterations
1 branch-and-cut nodes
plus 77 simplex iterations for intbasis


In case there is some demand unacounted for, we do one last iteration with a different model.<br>
The "gredy.mod" model does not handle this for performance and complexity reasons.<br>
We use the optimized model.<br>

In [28]:
# if total demand is not a multiple of the capacity, get the last 
if math.floor(sum(df["demand"].tolist())/50000) < sum(df["demand"].tolist())/50000:
    ampl = AMPL()
    ampl.option['solver'] = 'gurobi'
    # use the optimized model, so that we dont have to create a new one
    ampl.read("optimized.mod")
    #set the parameters
    ampl.param['j'] = len(df)
    ampl.param['k'] = len(df)
    ampl.param['capacity'] = 50000
    ampl.param['facilityCost'] = 100000
    ampl.param['demand'] = demand
    ampl.param['transportCost'] = transportCost
    ampl.solve()
    # add new cost to the total cost
    totalCost += ampl.getObjective('cost').value()
    # save the final facility
    x = ampl.getVariable("facility")
    for index, xi in x:
        if (xi.value()!=0):
            facilities.append(index)
    x = ampl.getVariable("transport")
    for index, xi in x:
        # if something got transported
        if (round(xi.value())!=0):
            totalTransport.append((index,round(xi.value())))
end = time.time()

Gurobi 9.0.0: optimal solution; objective 109101.2265
454 simplex iterations
1 branch-and-cut nodes
plus 331 simplex iterations for intbasis


Finally, we present the results. These were aggregated as the facilities were selected.<br>

In [29]:
print(end - start, " seconds taken")
print("Total cost: ", totalCost)
print("Facilities: ", facilities)
print("Transports: ")
for transport in totalTransport:
    print(transport[0],transport[1])

161.5383267402649  seconds taken
Total cost:  1006130.6360232439
Facilities:  [1.0, 2.0, 7.0, 146.0, 154.0, 348.0, 13.0, 25.0]
Transports: 
(1.0, 1.0) 25891
(1.0, 17.0) 2629
(1.0, 18.0) 2626
(1.0, 19.0) 1256
(1.0, 32.0) 1850
(1.0, 40.0) 1701
(1.0, 41.0) 1690
(1.0, 46.0) 1591
(1.0, 79.0) 1125
(1.0, 86.0) 1050
(1.0, 87.0) 1045
(1.0, 92.0) 1009
(1.0, 96.0) 974
(1.0, 98.0) 967
(1.0, 112.0) 886
(1.0, 120.0) 853
(1.0, 154.0) 637
(1.0, 201.0) 526
(1.0, 283.0) 386
(1.0, 302.0) 360
(1.0, 324.0) 325
(1.0, 326.0) 325
(1.0, 345.0) 298
(2.0, 2.0) 12482
(2.0, 9.0) 3541
(2.0, 21.0) 2499
(2.0, 29.0) 1971
(2.0, 34.0) 1822
(2.0, 44.0) 814
(2.0, 53.0) 1447
(2.0, 58.0) 1404
(2.0, 59.0) 1385
(2.0, 60.0) 1385
(2.0, 63.0) 1322
(2.0, 67.0) 1297
(2.0, 70.0) 1221
(2.0, 71.0) 1196
(2.0, 83.0) 1067
(2.0, 102.0) 948
(2.0, 110.0) 900
(2.0, 121.0) 851
(2.0, 127.0) 806
(2.0, 130.0) 793
(2.0, 139.0) 737
(2.0, 140.0) 723
(2.0, 141.0) 718
(2.0, 142.0) 713
(2.0, 151.0) 645
(2.0, 156.0) 628
(2.0, 159.0) 613
(2.0, 171.0) 5

### Multiple greedy

Another possible approach that was explored was greedy search with multiple facilities at a time. This means, instead of selecting the once facility that gets the best result on each iteration, we can select several.<br>
Using this approach we would expect to usually obtain better results.<br>

In [30]:
df = distanceDF

Much like before, we obtain  the costs taking into account both air and land distances.<br>

In [31]:
airports = ["Maia","Moscavide e Portela","Faro"]
airDistances = []
for j in range(0,len(df)):
    ds = []
    for i in le.transform(airports).tolist():
        d1 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,2]-df.iloc[j,2])
        d2 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,3]-df.iloc[j,3])
        ds.append(d1+d2)
    airDistances.append(min(ds)) 

In [32]:
airCost = 0.001
landCost = 0.01

transportCost = []
for i in range(0,len(df)):
    for j in range(0,len(df)):
        d1 = abs(df.iloc[i,2]-df.iloc[j,2])
        d2 = abs(df.iloc[i,3]-df.iloc[j,3])
        transportCost.append((d1+d2)*landCost+airDistances[i]*airCost) 

We start by using the greedy model, fixing the number of facilities at our intended amount.<br>
Once we no longer have enough demand for said amount we use the previous (single) greedy model.<br>
And finally, for the remaining data we use the optimized model.<br>

In [33]:
totalCost = 0
demand = df["demand"].tolist()
facilities = []
totalTransport = []
multi = 2
start = time.time()

iterator=0
while iterator < math.floor(sum(df["demand"].tolist())/50000):
    print(math.floor(sum(df["demand"].tolist())/50000),math.floor(sum(df["demand"].tolist())/(50000*multi)),iterator)
    # model needs to be reset after each itertion
    ampl = AMPL()
    ampl.option['solver'] = 'gurobi'
    
    # if enough data for multiple facilities, select them
    if iterator + multi <= math.floor(sum(df["demand"].tolist())/50000):
        iterator+=multi
        ampl.read("multiGreedy.mod")
        ampl.param['multi'] = multi
    # otherwise select only one
    else:
        iterator+=1
        ampl.read("greedy.mod")

    #set the parameters
    ampl.param['j'] = len(df)
    ampl.param['k'] = len(df)
    ampl.param['capacity'] = 50000
    ampl.param['facilityCost'] = 100000
    #only demand and transportCost change in each iteration
    ampl.param['demand'] = demand
    ampl.param['transportCost'] = transportCost
    #solve
    ampl.solve()
    # add new cost to the total cost
    totalCost += round(ampl.getObjective('cost').value())
    # get the facility list
    x = ampl.getVariable("facility")
    newFacilicies = []
    for index, xi in x:
        # if it is the selected facility
        if (xi.value()!=0):
            # save it
            newFacilicies.append(index)
            facilities.append(index)
    # get the thransport values
    x = ampl.getVariable("transport")
    for index, xi in x:
        # if something got transported
        if (round(xi.value())!=0):
            # reduce demand in tranport location -- ampl index starts at 1 but python at 0, reduce by 1
            demand[int(index[1])-1] -= round(xi.value()) #rounding needed due to rounding errors causing problems
            totalTransport.append((index,round(xi.value())))
    # for all transport combinations
    for i in range(0,len(df)):
        for j in range(0,len(df)):
            # if row is from the last selected facility
            if i+1 in newFacilicies:
                # change transport cost so that it will no longer transport
                transportCost[i*len(df)+j] = 99999

7 3 0
Gurobi 9.0.0: optimal solution; objective 207221.9598
2621 simplex iterations
1 branch-and-cut nodes
plus 185 simplex iterations for intbasis
7 3 2
Gurobi 9.0.0: optimal solution; objective 222031.3584
15705 simplex iterations
1 branch-and-cut nodes
plus 340 simplex iterations for intbasis
7 3 4
Gurobi 9.0.0: optimal solution; objective 255317.4471
153735 simplex iterations
1461 branch-and-cut nodes
plus 154 simplex iterations for intbasis
7 3 6
Gurobi 9.0.0: optimal solution; objective 186736.334
584 simplex iterations
1 branch-and-cut nodes
plus 73 simplex iterations for intbasis


In [34]:
# if total demand is not a multiple of the capacity, get the last 
if sum(df["demand"].tolist()) % (50000*multi) != 0:
    ampl = AMPL()
    ampl.option['solver'] = 'gurobi'
    # use the optimized model, so that we dont have to create a new one
    ampl.read("optimized.mod")
    #set the parameters
    ampl.param['j'] = len(df)
    ampl.param['k'] = len(df)
    ampl.param['capacity'] = 50000
    ampl.param['facilityCost'] = 100000
    ampl.param['demand'] = demand
    ampl.param['transportCost'] = transportCost
    ampl.solve()
    # add new cost to the total cost
    totalCost += round(ampl.getObjective('cost').value())
    # save the final facility
    x = ampl.getVariable("facility")
    for index, xi in x:
        if (xi.value()!=0):
            facilities.append(index)
    x = ampl.getVariable("transport")
    for index, xi in x:
        # if something got transported
        if (round(xi.value())!=0):
            totalTransport.append((index,round(xi.value())))
end = time.time()

Gurobi 9.0.0: optimal solution; objective 109130.1963
146 simplex iterations
1 branch-and-cut nodes
plus 256 simplex iterations for intbasis


In [35]:
print(end - start, " seconds taken")
print("Total cost: ", totalCost)
print("Facilities: ", facilities)
print("Transports: ")
for transport in totalTransport:
    print(transport[0],transport[1])
UB = totalCost

509.5612258911133  seconds taken
Total cost:  980436
Facilities:  [1.0, 2.0, 8.0, 79.0, 278.0, 344.0, 152.0, 328.0]
Transports: 
(1.0, 1.0) 25891
(1.0, 17.0) 2629
(1.0, 18.0) 2626
(1.0, 19.0) 1256
(1.0, 32.0) 1850
(1.0, 40.0) 1701
(1.0, 41.0) 1690
(1.0, 46.0) 1591
(1.0, 79.0) 1125
(1.0, 86.0) 1050
(1.0, 87.0) 1045
(1.0, 92.0) 1009
(1.0, 96.0) 974
(1.0, 98.0) 967
(1.0, 112.0) 886
(1.0, 120.0) 853
(1.0, 154.0) 637
(1.0, 201.0) 526
(1.0, 283.0) 386
(1.0, 302.0) 360
(1.0, 324.0) 325
(1.0, 326.0) 325
(1.0, 345.0) 298
(2.0, 2.0) 12482
(2.0, 9.0) 3541
(2.0, 21.0) 2499
(2.0, 29.0) 1971
(2.0, 34.0) 1822
(2.0, 44.0) 814
(2.0, 53.0) 1447
(2.0, 58.0) 1404
(2.0, 59.0) 1385
(2.0, 60.0) 1385
(2.0, 63.0) 1322
(2.0, 67.0) 1297
(2.0, 70.0) 1221
(2.0, 71.0) 1196
(2.0, 83.0) 1067
(2.0, 102.0) 948
(2.0, 110.0) 900
(2.0, 121.0) 851
(2.0, 127.0) 806
(2.0, 130.0) 793
(2.0, 139.0) 737
(2.0, 140.0) 723
(2.0, 141.0) 718
(2.0, 142.0) 713
(2.0, 151.0) 645
(2.0, 156.0) 628
(2.0, 159.0) 613
(2.0, 171.0) 585
(2.0, 17

We can see from the above results, that for this data in particular the method is slightly better than the regular greedy one, although, also much slower.<br>

### Transport problem

A related problem to the one from for faliclity location selection is the transport problem. This is a very similar problem, in which there are no costs for oppening facilities. It is also a substantially simpler problem to solve.<br>

In [36]:
df = distanceDF

In [37]:
ampl = AMPL()
ampl.option['solver'] = 'gurobi'
ampl.read("transport.mod")

In [38]:
ampl.param['j'] = len(df)
ampl.param['k'] = len(df)
ampl.param['capacity'] = 50000
ampl.param['demand'] = df["demand"].tolist()

In [39]:
airports = ["Maia","Moscavide e Portela","Faro"]
airDistances = []
for j in range(0,len(df)):
    ds = []
    for i in le.transform(airports).tolist():
        d1 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,2]-df.iloc[j,2])
        d2 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,3]-df.iloc[j,3])
        ds.append(d1+d2)
    airDistances.append(min(ds)) 

In [40]:
airCost = 0.001
landCost = 0.01

transportCost = []
for i in range(0,len(df)):
    for j in range(0,len(df)):
        d1 = abs(df.iloc[i,2]-df.iloc[j,2])
        d2 = abs(df.iloc[i,3]-df.iloc[j,3])
        transportCost.append((d1+d2)*landCost+airDistances[i]*airCost) 

In [41]:
ampl.param['transportCost'] = transportCost

In [42]:
start = time.time()
ampl.solve()
end = time.time()
print(end - start, " seconds taken")

Gurobi 9.0.0: optimal solution; objective 20154.24529
0.913649320602417  seconds taken


We can see above that the cost is way lower than for all previous methods. However, this would obviously be the case, as facility costs are not being taken into consideration.<br>
Below we can see that each DC send to its own location, with the land transport cost being 0. As such, the cost presented comes exclusively from air travel.<br>

In [43]:
print("Land Transports - Inactive ones not shown.")
x = ampl.getVariable("transport")
for index, xi in x:
    if (round(xi.value())!=0):
        print(index,round(xi.value()))
LB = round(ampl.getObjective('cost').value()) + 100000*math.ceil(sum(df["demand"].tolist())/50000)

Land Transports - Inactive ones not shown.
(1.0, 1.0) 25891
(2.0, 2.0) 12482
(3.0, 3.0) 8943
(4.0, 4.0) 6070
(5.0, 5.0) 5856
(6.0, 6.0) 5330
(7.0, 7.0) 5170
(8.0, 8.0) 4700
(9.0, 9.0) 3541
(10.0, 10.0) 3313
(11.0, 11.0) 3312
(12.0, 12.0) 2904
(13.0, 13.0) 2781
(14.0, 14.0) 2735
(15.0, 15.0) 2732
(16.0, 16.0) 2709
(17.0, 17.0) 2629
(18.0, 18.0) 2626
(19.0, 19.0) 2564
(20.0, 20.0) 2500
(21.0, 21.0) 2499
(22.0, 22.0) 2336
(23.0, 23.0) 2330
(24.0, 24.0) 2256
(25.0, 25.0) 2234
(26.0, 26.0) 2068
(27.0, 27.0) 2057
(28.0, 28.0) 2031
(29.0, 29.0) 1971
(30.0, 30.0) 1947
(31.0, 31.0) 1925
(32.0, 32.0) 1850
(33.0, 33.0) 1822
(34.0, 34.0) 1822
(35.0, 35.0) 1808
(36.0, 36.0) 1750
(37.0, 37.0) 1738
(38.0, 38.0) 1732
(39.0, 39.0) 1719
(40.0, 40.0) 1701
(41.0, 41.0) 1690
(42.0, 42.0) 1674
(43.0, 43.0) 1666
(44.0, 44.0) 1640
(45.0, 45.0) 1606
(46.0, 46.0) 1591
(47.0, 47.0) 1508
(48.0, 48.0) 1501
(49.0, 49.0) 1489
(50.0, 50.0) 1470
(51.0, 51.0) 1462
(52.0, 52.0) 1453
(53.0, 53.0) 1447
(54.0, 54.0) 1446
(

### Optimization 2

With some new information gained from the previous models, we once again tackle the problem of obtaining the exact best solution.<br>
For this, we use the cost values from the multiple greedy model and the transport model. These give us a lower and upper bound on cost, allowing us to restrict the search. With them, we can also restrict the number of active facilities, restricting as such, the search space even more. One final constraint that was discarded for the first optimized model as it did not improve performance, the transport from each facility to a customer not exceeding the capacity, was added.<br>

In [73]:
df = distanceDF

In [74]:
ampl = AMPL()
ampl.option['solver'] = 'gurobi'
ampl.read("optimized_2.mod")

In [75]:
ampl.param['j'] = len(df)
ampl.param['k'] = len(df)
ampl.param['capacity'] = 50000
ampl.param['facilityCost'] = 100000
ampl.param['demand'] = df["demand"].tolist()

The new upper and lower bounds are added to the model, with no further parameter or code modifications from the first optimized model.<br>

In [76]:
ampl.param['LB'] = LB
ampl.param['UB'] = UB

In [77]:
airports = ["Maia","Moscavide e Portela","Faro"]
airDistances = []
for j in range(0,len(df)):
    ds = []
    for i in le.transform(airports).tolist():
        d1 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,2]-df.iloc[j,2])
        d2 = abs(distanceDF[distanceDF["locationLabel"]==i].iloc[0,3]-df.iloc[j,3])
        ds.append(d1+d2)
    airDistances.append(min(ds)) 

In [78]:
airCost = 0.001
landCost = 0.01

transportCost = []
for i in range(0,len(df)):
    for j in range(0,len(df)):
        d1 = abs(df.iloc[i,2]-df.iloc[j,2])
        d2 = abs(df.iloc[i,3]-df.iloc[j,3])
        transportCost.append((d1+d2)*landCost+airDistances[i]*airCost) 

In [79]:
start = time.time()
ampl.param['transportCost'] = transportCost

In [80]:
ampl.solve()
end = time.time()
print(end - start, " seconds taken")

Gurobi 9.0.0: optimal solution; objective 944010.834
4024647 simplex iterations
7228 branch-and-cut nodes
plus 9787 simplex iterations for intbasis
4849.119736433029  seconds taken


In [81]:
print("Facilities - Inactive ones not shown.")
x = ampl.getVariable("facility")
for index, xi in x:
    if (xi.value()!=0):
        print(index,xi.value())
print("Land Transports - Inactive ones not shown.")
x = ampl.getVariable("transport")
for index, xi in x:
    if (round(xi.value())!=0):
        print(index,round(xi.value()))

Facilities - Inactive ones not shown.
1.0 1.0
2.0 1.0
8.0 1.0
19.0 1.0
20.0 1.0
184.0 1.0
278.0 1.0
369.0 1.0
Land Transports - Inactive ones not shown.
(1.0, 1.0) 25891
(1.0, 11.0) 3312
(1.0, 15.0) 2732
(1.0, 32.0) 1850
(1.0, 41.0) 1690
(1.0, 46.0) 1591
(1.0, 73.0) 1185
(1.0, 76.0) 1163
(1.0, 79.0) 521
(1.0, 87.0) 1045
(1.0, 92.0) 1009
(1.0, 98.0) 967
(1.0, 112.0) 886
(1.0, 114.0) 877
(1.0, 120.0) 853
(1.0, 137.0) 745
(1.0, 154.0) 637
(1.0, 197.0) 536
(1.0, 201.0) 526
(1.0, 225.0) 379
(1.0, 302.0) 360
(1.0, 324.0) 325
(1.0, 326.0) 325
(1.0, 345.0) 298
(1.0, 346.0) 297
(2.0, 2.0) 12482
(2.0, 9.0) 3541
(2.0, 21.0) 2499
(2.0, 29.0) 1971
(2.0, 34.0) 1822
(2.0, 52.0) 679
(2.0, 53.0) 1447
(2.0, 58.0) 1404
(2.0, 59.0) 1385
(2.0, 60.0) 1385
(2.0, 61.0) 1383
(2.0, 63.0) 1322
(2.0, 67.0) 1297
(2.0, 70.0) 1221
(2.0, 71.0) 1196
(2.0, 83.0) 1067
(2.0, 102.0) 948
(2.0, 110.0) 900
(2.0, 127.0) 806
(2.0, 130.0) 793
(2.0, 134.0) 761
(2.0, 139.0) 737
(2.0, 140.0) 723
(2.0, 141.0) 718
(2.0, 142.0) 713
(

As we can see above, this model takes about two hours to run in this machine. This is substantially slower than the non-exact models, with only a slight reduction to cost.<br>

### Project Specifics

We were asked to answer a few specific questions. Those will be analised and answered now.<br>

The first one is to identify the location of each DC.<br>

In [82]:
print("The locations are:")
x = ampl.getVariable("facility")
for index, xi in x:
    if (xi.value()!=0):
        print(le.inverse_transform([distanceDF["locationLabel"][index-1]]))

The locations are:
['Lisbon']
['Porto']
['Cacem']
['Barreiro']
['Monsanto']
['Sao Bras de Alportel']
['Joane']
['Sao Roque']


The second one is to identify the town with the largest delivery costs.<br>

In [83]:
d = {}
x = ampl.getVariable("transport")
for index, xi in x:
    if (round(xi.value())!=0):
        if index[1] in d.keys():
            d[index[1]] += round(xi.value() * transportCost[round(index[0]*len(df)+index[1])],2)
        else:
            d[index[1]] = round(xi.value() * transportCost[round(index[0]*len(df)+index[1])],2)
maximum = (0,0)
for key in d.keys():
    if d[key] > maximum[1]:
        maximum = (key,d[key])
print("The town with the largets delivery costs is:")
print(le.inverse_transform([distanceDF["locationLabel"][maximum[0]-1]]),":",maximum[1])

The town with the largets delivery costs is:
['Coimbra'] : 20904.14


# Conclusions

The presented problem is, at first look, surprisingly complicated. With such a simple optimization problem to define and a low amount of data, we would expect to have no difficulties. However, of further inspection, the true nature of the problem arises, as an exponential problem of high degree.<br>

Finding the best solution for this problem is extremely difficult, and with a higher amount of data might not be feasible.<br>
Three of the tried approaches have their usefulness, with different strengths and weaknesses.<br>

The first of these, greedy search, is capable of obtaining results quite close to the optimum in a very low amount of time and great scalability. For problems of higher dimension, this one might be the best bet.<br>

The second one, multiple greedy search, is placed in the middle of the three models, presenting overall better results, but also worse performance, taking substantially longer. It is also not as scalable as the first model.<br>

The third one, always obtains the best possible answer, however, this comes with a high computational cost.<br>

For this particular problem, in an industrial setting, two hours to compute, in order to increase the profit of margin even slightly is perfectly acceptable, with the last model being the best one. However, with an increase in data sizes, quickly would this model have to be changed to the second one, and later to the first one.<br>
The first two models are of particular use, when the costs, demands, etc. are only estimates, and we would be interested in trying different scenarios, allowing a broader range of scenarios to be tested quickly, allowing overall for better decisions to be made over what little improvement in accuracy the final model would give.<br>